To start running this notebook: in a shell, go to the home directory of the github repo.
```
cd src
conda create geocache python=3.11
conda activate geocache
conda install numpy jupyter
jupyter notebook usd-separate.ipynb
```
Then you can shift-enter to run each cell here.

To run the Alembic parts you'll need to have run the Running Alembic part of the [README](https://github.com/imgspc/geocache-compression/blob/main/README.md#running-alembic).

In [ ]:
# Make sure we have USD installed.
!pip install usd-core

In [ ]:
# Settings start here. Evaluate this cell whenever needed.
from pathlib import Path

home = Path.home()
infile = str(home / "projects/ALab/ALab/entry.usda")
outdir = str(home / "projects/geocache-compression/build")
octopus = str(home / "projects/geocache-compression/build/octopus.json")
octopus_bindir = home / "projects/geocache-compression/build"

kmeans = False
clustersize = 1000
quality = 0.99

########################################
from pxr import Usd
from embedding import clustering
from embedding.io import separate_usd
from embedding.io import create_embedding
from embedding.io import read_embedding
from embedding.metric import Report
from embedding.io import read_binfile, parse_json_file, run_all_reports
import os
import math

if kmeans:
    cluster_fn = clustering.cluster_kmeans
else:
    cluster_fn = clustering.cluster_by_index

In [ ]:
# Load the files.
stage = Usd.Stage.Open(infile)
mpu = stage.GetMetadata("metersPerUnit")
print(mpu)

package = separate_usd(infile, outdir, verbose=False)
print(f"{package.inputfile} has {len(package.headers)} animations")

In [ ]:
# Test just doing one embedding.
header = max(package.headers, key=lambda h: h.numbytes())  # compress the biggest file
# header = package.get_header("/root/remi/head_M_hrc/GEO/head_M_hrc/facePlate_M_geo")
print(f"{header}")
files = create_embedding(
    header,
    cluster_fn=cluster_fn,
    quality=quality,
    cluster_size=clustersize,
    verbose=True,
)

print(files)

postdata = read_embedding(header, files, verbose=True)
predata = read_binfile(header)
report = Report(predata, postdata, sum(os.path.getsize(path) for path in files))
print(f"{header.path}")
report.print_report(mpu)
for path in files:
    print(f"  {os.path.getsize(path)} {path}")

In [ ]:
# Test *all* the animations.
reports = run_all_reports(
    package,
    quality=quality,
    cluster_fn=cluster_fn,
    cluster_size=clustersize,
    verbose=True,
)
print("done")

In [ ]:
# Play around with the overall set of reports.
orig = sum(r.original_size for r in reports.values())
compressed = sum(r.compressed_size for r in reports.values())
print(f"{orig} vs {compressed} -- {1-compressed/orig:.2%} compression")

h = max(r.hausdorff for r in reports.values()) * mpu
linf = max(r.Linf for r in reports.values()) * mpu
print(f"Max error: {h} m distance, {linf} m any single coordinate")

corrected = max(r.corrected_Linf for r in reports.values()) * mpu
uncorrected = sum(r.numuncorrectable for r in reports.values())
numvalues = sum(r.original_numvalues for r in reports.values())
print(f"{uncorrected} uncorrectable out of {numvalues} ({uncorrected/numvalues:%})")

In [ ]:
############ Alembic fun times
abc_package = parse_json_file(octopus)
for h in abc_package.headers:
    if not os.path.isabs(h.binpath):
        h.binpath = str(octopus_bindir / h.binpath)

mpu = 0.01
reports = run_all_reports(
    abc_package,
    cluster_fn=cluster_fn,
    cluster_size=clustersize,
    quality=quality,
    verbose=True,
)